In [1]:
!python -m pip install --upgrade pip
!pip install contextualized-topic-models
!python -m spacy download fr_core_news_md



     |████████████████████████████████| 46.9 MB 44.0 MB/s            


  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
  Attempting uninstall: smart-open
    Found existing installation: smart-open 2.2.1
    Uninstalling smart-open-2.2.1:
      Successfully uninstalled smart-open-2.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gretel-client 0.8.5 requires smart-open<3.0,>=2.1.0, but you have smart-open 5.2.1 which is incompatible.
flair 0.7 requires transformers<=3.5.1,>=3.5.0, but you have transformers 4.15.0 which is incompatible.
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')


In [2]:
from contextualized_topic_models.models.ctm import ZeroShotTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.data_preparation import bert_embeddings_from_file

from datasets import load_dataset
from bertopic import BERTopic
import pandas as pd 
from nltk.corpus import stopwords
import nltk
import re
from nltk.stem.snowball import SnowballStemmer
import spacy


piaf_dataset = load_dataset('piaf', split="train")
df_piaf = piaf_dataset.to_pandas()
text_for_contextual = list(df_piaf.context.unique())





Reusing dataset piaf (/Users/ytadjota/.cache/huggingface/datasets/piaf/plain_text/1.0.0/535c60f4155fe1d644c1746e86131963c082f309d0dbb5ba4d606786c7f4a6ae)


In [10]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ytadjota/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
min_topic_length = 3
stemmer = SnowballStemmer(language='french')
spacy_stemmer = spacy.load('fr_core_news_md')
final_stopwords_list = stopwords.words('english') + stopwords.words('french')

def tokenize_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = [token for token in tokens if token not in final_stopwords_list]
    stems = [spacy_stemmer(t)[0].lemma_ for t in filtered_tokens if (len(t) > min_topic_length and re.search('[a-zA-Z]', t))]
    return stems

text_for_bow = [" ".join(tokenize_and_stem(context)) for context in text_for_contextual]



In [ ]:
qt = TopicModelDataPreparation("paraphrase-multilingual-mpnet-base-v2")

training_dataset = qt.fit(text_for_contextual=text_for_contextual, text_for_bow=text_for_bow)

ctm = ZeroShotTM(bow_size=len(qt.vocab), contextual_size=768, n_components=50)

ctm.fit(training_dataset) # run the model


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [6]:
ctm.get_topics(20)

defaultdict(list,
            {0: ['rencontre',
              'lucifer',
              'faire',
              'perdre',
              'arrive',
              'dieu',
              'dernier',
              'tard',
              'que',
              'fait',
              'parvenir',
              'décide',
              'tuer',
              'hôte',
              'remporte',
              'rendre',
              'bond',
              'laisser',
              'tournoi',
              'corinne'],
             1: ['britannique',
              'armée',
              'troupe',
              'général',
              'bataille',
              'juillet',
              'américain',
              'homme',
              'soldat',
              'prise',
              'force',
              'défaite',
              'attaquer',
              'battre',
              'guerre',
              'mener',
              'grande',
              'pacha',
              'fort',
              'commander'],
        